# **Importings**


## Libraries

In [ ]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Input, BatchNormalization, Concatenate, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K

from keras.models import load_model


import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 110
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer
from sklearn.neighbors import KNeighborsClassifier
from tabulate import tabulate

## Data

In [ ]:
def addresses_generator(signal, loads, states, reps):
    results = {}
    for load in loads:
        results[load] = []
        for state in states:
            results[load].append([signal + '_' + state +'_torque' + load + '_' + rep for rep in reps])

    return results

In [ ]:
def df_loader_from_addresses(addresses, base_dir, signal):
    results = {}
    for load in list(addresses.keys()):
        results[load] = []
        for addresses_of_same_load_state_reps in addresses[load]:
            list_of_dfs = [pd.read_hdf(base_dir + signal+ '//' + address + '.hdf', address) for address in addresses_of_same_load_state_reps]
            concat_dfs = pd.concat(list_of_dfs)
            concat_dfs = concat_dfs.reset_index(drop=True)
            results[load].append(concat_dfs)
        results[load] = pd.concat(results[load])
        results[load] = results[load].reset_index(drop=True)
    return results

In [ ]:
# Set the path to the base directory of frequency-domain hdf files
base_dir = ''

In [ ]:
loads = ['05', '20', '25', '40']
reps = ['0', '1']
states = ['r1b', 'r2b', 'r3b', 'r4b', 'rs']

In [ ]:
Ia_addresses = addresses_generator('Ia', loads, states, reps)
Ib_addresses = addresses_generator('Ib', loads, states, reps)
Ic_addresses = addresses_generator('Ic', loads, states, reps)

In [ ]:
Ia_dict = df_loader_from_addresses(Ia_addresses, base_dir, 'Ia')
Ib_dict = df_loader_from_addresses(Ib_addresses, base_dir, 'Ib')
Ic_dict = df_loader_from_addresses(Ic_addresses, base_dir, 'Ic')

# **Data Preparation and Preprocessing**

In [ ]:
def train_test_splitter(signal_dict):
  signal_train = {}
  signal_test = {}

  for load in list(signal_dict.keys()):
    temp_train, temp_test = train_test_split(signal_dict[load], test_size=0.25, random_state=42)
    
    signal_train[load] = temp_train
    signal_test[load] = temp_test

  return signal_train, signal_test

In [ ]:
def train_test_scaler(signal_train, signal_test):

  for load in list(signal_train.keys()):
    temp_scaler = MinMaxScaler()
    temp_scaler.fit(signal_train[load])
    temp_train_scaled = temp_scaler.transform(signal_train[load])
    temp_test_scaled = temp_scaler.transform(signal_test[load])

    signal_train[load] = temp_train_scaled
    signal_test[load] = temp_test_scaled

  return signal_train, signal_test

In [ ]:
def pair_creator_limited(vectors, labels, pairs_for_each_anchor):

  pair_of_vectors = []
  labels_of_pairs = []

  idx = {i: np.where(labels == i)[0] for i in np.unique(labels)}
  for idxA in range(len(vectors)):
    current_vector = vectors[idxA]
    current_label = labels[idxA]

  # positive pairs

    if np.setdiff1d(idx[current_label], np.array(idxA)).shape[0] > 0:
      for i in range(int(pairs_for_each_anchor/2)):
        idxB = np.random.choice(np.setdiff1d(np.where(labels == current_label)[0], np.array(idxA)))
        pos_vector = vectors[idxB]
        pair_of_vectors.append([current_vector, pos_vector])
        labels_of_pairs.append([1])

    # Negative Pairs
    for j in range(int(pairs_for_each_anchor/2)):
      negIdx = np.where(labels != current_label)[0]
      neg_vector = vectors[np.random.choice(negIdx)]

      pair_of_vectors.append([current_vector, neg_vector])
      labels_of_pairs.append([0])

  return (np.array(pair_of_vectors), np.array(labels_of_pairs))

## Defining Labels

In [ ]:
labels = np.concatenate([
    np.full((300), 1),
    np.full((300), 2),
    np.full((300), 3),
    np.full((300), 4),
    np.full((300), 0),
])

## Train/Test Splitting

In [ ]:
Ia_train, Ia_test = train_test_splitter(Ia_dict)
Ib_train, Ib_test = train_test_splitter(Ib_dict)
Ic_train, Ic_test = train_test_splitter(Ic_dict)
labels_train , labels_test = train_test_split(labels, test_size=0.25, random_state=42)

In [ ]:
labels_train_dict = dict(zip(loads, [labels_train, labels_train, labels_train, labels_train]))
labels_test_dict = dict(zip(loads, [labels_test, labels_test, labels_test, labels_test]))

## Defining Mixed Load

In [ ]:
Ia_train['total'] = np.concatenate([Ia_train[load] for load in list(Ia_train.keys())])
Ia_test['total'] = np.concatenate([Ia_test[load] for load in list(Ia_test.keys())])

Ib_train['total'] = np.concatenate([Ib_train[load] for load in list(Ib_train.keys())])
Ib_test['total'] = np.concatenate([Ib_test[load] for load in list(Ib_test.keys())])

Ic_train['total'] = np.concatenate([Ic_train[load] for load in list(Ic_train.keys())])
Ic_test['total'] = np.concatenate([Ic_test[load] for load in list(Ic_test.keys())])

labels_train_dict['total'] = np.concatenate([labels_train_dict[load] for load in list(labels_train_dict.keys())])
labels_test_dict['total'] = np.concatenate([labels_test_dict[load] for load in list(labels_test_dict.keys())])

## Feature Scaling

In [ ]:
Ia_train_scaled, Ia_test_scaled = train_test_scaler(Ia_train, Ia_test)
Ib_train_scaled, Ib_test_scaled = train_test_scaler(Ib_train, Ib_test)
Ic_train_scaled, Ic_test_scaled = train_test_scaler(Ic_train, Ic_test)

## Horizontal Concatanation of Currents

In [ ]:
I_train = np.concatenate((Ia_train_scaled['total'], Ib_train_scaled['total'], Ic_train_scaled['total']), axis=1)
I_test = np.concatenate((Ia_test_scaled['total'], Ib_test_scaled['total'], Ic_test_scaled['total']), axis=1)

## Regular/FewShot Train Splitting

In [ ]:
I_train_regular, I_train_fewShot = train_test_split(I_train, test_size=0.25, random_state=42)
labels_train_regular, labels_train_fewShot = train_test_split(labels_train_dict['total'], test_size=0.25, random_state=42)

## Binarizing Regular Labels

In [ ]:
labels_train_regular_bin = LabelBinarizer().fit_transform(y = labels_train_regular)
labels_test_bin = LabelBinarizer().fit_transform(y = labels_test_dict['total'])

## Creating Pairs for Few Shot Learning

In [ ]:
del Ia_train_scaled, Ia_test_scaled, Ib_train_scaled, Ib_test_scaled, Ic_train_scaled, Ic_test_scaled
del Ia_dict, Ib_dict, Ic_dict

In [ ]:
pair_of_vectors_train, pair_labels_train = pair_creator_limited(I_train_fewShot, labels_train_fewShot, 4)

# **Hybrid Model Training**

In [ ]:
class SaveBestModel(tf.keras.callbacks.Callback):
    def __init__(self, save_best_metric='val_loss', this_max=False):
        self.save_best_metric = save_best_metric
        self.max = this_max
        if this_max:
            self.best = float('-inf')
        else:
            self.best = float('inf')

    def on_epoch_end(self, epoch, logs=None):
        metric_value = logs[self.save_best_metric]
        if self.max:
            if metric_value > self.best:
                self.best = metric_value
                self.best_weights = self.model.get_weights()

        else:
            if metric_value < self.best:
                self.best = metric_value
                self.best_weights= self.model.get_weights()

In [ ]:
def fe_creator():

  input = Input(shape=9999, name='input1')
  input_emb = Dense(units=7500, activation='tanh', name = 'HL1_1')(input)
  input_emb = Dense(units=6000, activation='tanh', name = 'HL1_2')(input_emb)
  input_emb = Dense(units=4500, activation='tanh', name = 'HL1_3')(input_emb)
  input_emb = Dense(units=3000, activation='tanh', name = 'HL1_4')(input_emb)
  input_emb = Dense(units=1500, activation='tanh', name = 'HL1_5')(input_emb)
  input_emb = Dense(units=750, activation='tanh', name = 'HL1_6')(input_emb)
  input_emb = Dense(units=500, activation='tanh', name = 'HL1_7')(input_emb)
  input_emb = Dense(units=250, activation='tanh', name = 'HL1_8')(input_emb)
  input_emb = Dense(units=50, activation='tanh', name = 'HL1_9')(input_emb)
  
  return Model(inputs = input, outputs = input_emb)

In [ ]:
def euclidean_distance(vectors):
	(featsA, featsB) = vectors
	sumSquared = K.sum(K.square(featsA - featsB), axis=1, keepdims=True)
	return K.sqrt(K.maximum(sumSquared, K.epsilon()))

In [ ]:
def contrastive_loss(y, preds, margin=1):
	y = tf.cast(y, preds.dtype)
	squaredPreds = K.square(preds)
	squaredMargin = K.square(K.maximum(margin - preds, 0))
	loss = K.mean(y * squaredPreds + (1 - y) * squaredMargin)

	return loss

In [ ]:
def siamese_network_creator(base_model):

  input_shape = base_model.get_layer(index=0).input.shape[1]

  input_a = Input(shape=input_shape, name='Input_A')
  input_b = Input(shape=input_shape, name='Input_B')

  embedding_a = base_model(input_a)
  embedding_b = base_model(input_b)

  distance = Lambda(euclidean_distance)([embedding_a, embedding_b])

  model = Model(inputs=[input_a, input_b], outputs=distance)

  return model

In [ ]:
def knn_fitter(x_train, y_train, x_test, y_test, knn_neighbors):
  neigh_encoder_classifier = KNeighborsClassifier(n_neighbors=knn_neighbors)
  neigh_encoder_classifier.fit(x_train, y_train)

  return {
      'train_acc': neigh_encoder_classifier.score(x_train, y_train),
      'test_acc': neigh_encoder_classifier.score(x_test, y_test),
      }


In [ ]:
def model_freezer(model, layers_to_keep_trainable):
  for layer in model.layers[:-layers_to_keep_trainable]:
    layer.trainable = False

  return model

In [ ]:
def classifier_creator(num_classes):
  classifier = Dense(num_classes, activation='softmax', name='classification')

  return classifier

In [ ]:
def model_combiner(feature_extractor, classifier):
  feature_extractor_input = feature_extractor.get_layer(index=0).input

  return Model(feature_extractor_input, classifier(feature_extractor(feature_extractor_input)))

## Raw Data State Evaluation

In [ ]:
knn_neighbors = 5
raw_data_evaluation = knn_fitter(I_train, labels_train_dict['total'], I_test, labels_test_dict['total'], knn_neighbors)

print('Raw Data KNN Classification Results:', '\n',
        tabulate(raw_data_evaluation.items(), headers = ['Metric', 'Score']), '\n')

## FewShot Pre-training

In [ ]:
few_shot_learning_epochs = 100
few_shot_learning_rate = 0.00001

fe = fe_creator()
siamese = siamese_network_creator(fe)
opt_siamese = optimizers.Adam(learning_rate=few_shot_learning_rate, decay=few_shot_learning_rate / few_shot_learning_epochs)
siamese.compile(loss=contrastive_loss, optimizer=opt_siamese)
siamese_net_save_best_model = SaveBestModel()
fewShot_history = siamese.fit([pair_of_vectors_train[:, 0], pair_of_vectors_train[:, 1]], pair_labels_train[:],
                              validation_split=0.25,
                              epochs=few_shot_learning_epochs,
                              callbacks=[siamese_net_save_best_model])

siamese.set_weights(siamese_net_save_best_model.best_weights)

In [ ]:
plt.plot(fewShot_history.history['loss'])
plt.plot(fewShot_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
knn_neighbors = 5

I_train_FSL_embedding = fe.predict(I_train)
I_test_FSL_embedding = fe.predict(I_test)

fewShot_evaluation = knn_fitter(I_train_FSL_embedding, labels_train_dict['total'], I_test_FSL_embedding, labels_test_dict['total'], knn_neighbors)

print('FewShot Feature Extractor KNN Classification Results:', '\n',
        tabulate(fewShot_evaluation.items(), headers = ['Metric', 'Score']), '\n')

## Softmax Post-training

In [ ]:
lr = 0.000001
ep = 400


# fe = model_freezer(fe, 5)
softmax_classifier = classifier_creator(5)
model = model_combiner(fe, softmax_classifier)
best_model = SaveBestModel()
opt = optimizers.Adam(learning_rate=lr, decay=lr / ep)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

temp_histories  = model.fit(I_train_regular, labels_train_regular_bin, validation_split=0.25, epochs = ep, callbacks=[best_model])

model.set_weights(best_model.best_weights)

In [ ]:
plt.plot(temp_histories.history['loss'])
plt.plot(temp_histories.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(temp_histories.history['accuracy'])
plt.plot(temp_histories.history['val_accuracy'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
knn_neighbors = 5

I_train_hybrid_embedding = fe.predict(I_train)
I_test_hybrid_embedding = fe.predict(I_test)

fewShot_evaluation = knn_fitter(I_train_hybrid_embedding, labels_train_dict['total'],
                                I_test_hybrid_embedding, labels_test_dict['total'], knn_neighbors)

print('Hybrid Model KNN Classification Results:', '\n',
        tabulate(fewShot_evaluation.items(), headers = ['Metric', 'Score']), '\n')

In [ ]:
print('Hybrid Model Softmax Performance Results:', '\n',
      {'train_acc': model.evaluate(I_train, LabelBinarizer().fit_transform(labels_train_dict['total']))[1],
       'test_acc': model.evaluate(I_test, labels_test_bin)[1]})

